Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. 

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.7.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training
You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The tf.keras.callbacks.ModelCheckpoint callback allows you to continually save the model both during and at the end of training.

### Checkpoint callback usage
Create a tf.keras.callbacks.ModelCheckpoint callback that saves weights only during training:

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
23/32 [====================>.........] - ETA: 0s - loss: 1.3098 - sparse_categorical_accuracy: 0.6413
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 15ms/step - loss: 1.1449 - sparse_categorical_accuracy: 0.6790 - val_loss: 0.7054 - val_sparse_categorical_accuracy: 0.7840
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4176 - sparse_categorical_accuracy: 0.8760
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.4166 - sparse_categorical_accuracy: 0.8760 - val_loss: 0.6129 - val_sparse_categorical_accuracy: 0.8030
Epoch 3/10
24/32 [=====================>........] - ETA: 0s - loss: 0.3241 - sparse_categorical_accuracy: 0.9102
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.3189 - sparse_categorical_accuracy: 0.9130 - val_loss: 0.4867 - val_sparse_categorical_accuracy: 0.8500
Epoch 4/10
23

In [6]:
%ls training_1/

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [7]:
%ls training_1/checkpoint

training_1/checkpoint


As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3537 - sparse_categorical_accuracy: 0.0620 - 180ms/epoch - 6ms/step
Untrained model, accuracy:  6.20%


Then load the weights from the checkpoint and re-evaluate:

In [9]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4119 - sparse_categorical_accuracy: 0.8660 - 89ms/epoch - 3ms/step
Restored model, accuracy: 86.60%


### Checkpoint callback options  
  The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [10]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=20, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt


In [12]:
%ls training_2/

checkpoint                        cp-0010.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0015.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0015.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0020.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0020.ckpt.index
cp-0010.ckpt.data-00000-of-00001


**What are these files?**  
  The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

One or more shards that contain your model's weights.
An index file that indicates which weights are stored in which shard.
If you are training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0020.ckpt'

To test, reset the model and load the latest checkpoint:

In [14]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4493 - sparse_categorical_accuracy: 0.8690 - 173ms/epoch - 5ms/step
Restored model, accuracy: 86.90%


**Manually save weights**  
Manually saving weights with the Model.save_weights method. By default, tf.keras—and save_weights in particular—uses the TensorFlow checkpoint format with a .ckpt extension (saving in HDF5 with a .h5 extension is covered in the Save and serialize models guide):

In [15]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4493 - sparse_categorical_accuracy: 0.8690 - 168ms/epoch - 5ms/step
Restored model, accuracy: 86.90%


In [19]:
%ls checkpoints/

checkpoint  my_checkpoint.data-00000-of-00001  my_checkpoint.index


**Save the entire model**  
  Call model.save to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

**An entire model can be saved in two different file formats (SavedModel and HDF5). The TensorFlow SavedModel format is the default file format in TF2.x.**  However, models can be saved in HDF5 format. More details on saving entire models in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (Saved Model, HDF5) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (Saved Model, HDF5)

*Custom objects (e.g. subclassed models or layers) require special attention when saving and loading. See the Saving custom objects section below

**SavedModel format**  
  The SavedModel format is another way to serialize models. Models saved in this format can be restored using tf.keras.models.load_model and are compatible with TensorFlow Serving. The SavedModel guide goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to save and restore the model.

In [27]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=20)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/20
32/32 [==============================] - 0s 5ms/step - loss: 1.2132 - sparse_categorical_accuracy: 0.6490
Epoch 2/20
32/32 [==============================] - 0s 4ms/step - loss: 0.4327 - sparse_categorical_accuracy: 0.8790
Epoch 3/20
32/32 [==============================] - 0s 5ms/step - loss: 0.3012 - sparse_categorical_accuracy: 0.9190
Epoch 4/20
32/32 [==============================] - 0s 4ms/step - loss: 0.2215 - sparse_categorical_accuracy: 0.9390
Epoch 5/20
32/32 [==============================] - 0s 5ms/step - loss: 0.1691 - sparse_categorical_accuracy: 0.9570
Epoch 6/20
32/32 [==============================] - 0s 5ms/step - loss: 0.1151 - sparse_categorical_accuracy: 0.9820
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0853 - sparse_categorical_accuracy: 0.9880
Epoch 8/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0647 - sparse_categorical_accuracy: 0.9950
Epoch 9/20
32/32 [==============================] - 0s 5ms/step 

In [28]:
%ls saved_model/my_model/

assets/  keras_metadata.pb  saved_model.pb  variables/


In [29]:
%ls saved_model/my_model/variables/

variables.data-00000-of-00001  variables.index


Reload a fresh Keras model from the saved model:

In [30]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4212 - sparse_categorical_accuracy: 0.8760 - 163ms/epoch - 5ms/step
Restored model, accuracy: 87.60%
(1000, 10)


### HDF5 format
Keras provides a basic save format using the HDF5 standard.

In [32]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=20)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/20
32/32 [==============================] - 0s 5ms/step - loss: 1.2139 - sparse_categorical_accuracy: 0.6600
Epoch 2/20
32/32 [==============================] - 0s 5ms/step - loss: 0.4300 - sparse_categorical_accuracy: 0.8800
Epoch 3/20
32/32 [==============================] - 0s 5ms/step - loss: 0.2848 - sparse_categorical_accuracy: 0.9300
Epoch 4/20
32/32 [==============================] - 0s 6ms/step - loss: 0.2164 - sparse_categorical_accuracy: 0.9550
Epoch 5/20
32/32 [==============================] - 0s 5ms/step - loss: 0.1569 - sparse_categorical_accuracy: 0.9680
Epoch 6/20
32/32 [==============================] - 0s 5ms/step - loss: 0.1162 - sparse_categorical_accuracy: 0.9740
Epoch 7/20
32/32 [==============================] - 0s 6ms/step - loss: 0.0842 - sparse_categorical_accuracy: 0.9870
Epoch 8/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0693 - sparse_categorical_accuracy: 0.9920
Epoch 9/20
32/32 [==============================] - 0s 5ms/step 

In [33]:
%ls 

checkpoints/  my_model.h5  sample_data/  saved_model/  training_1/  training_2/


Now, recreate the model from that file:

In [34]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [35]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4305 - sparse_categorical_accuracy: 0.8720 - 170ms/epoch - 5ms/step
Restored model, accuracy: 87.20%


Keras saves models by inspecting their architectures. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration (what you pass to the .compile() method)
*  The optimizer and its state, if any (this enables you to restart training where you left off)

## https://www.tensorflow.org/guide/keras/save_and_serialize#saving_the_architecture

In [36]:
model.get_config()

{'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 784),
    'dtype': 'float32',
    'name': 'dense_14_input',
    'ragged': False,
    'sparse': False}},
  {'class_name': 'Dense',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'batch_input_shape': (None, 784),
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'kernel_regularizer': None,
    'name': 'dense_14',
    'trainable': True,
    'units': 512,
    'use_bias': True}},
  {'class_name': 'Dropout',
   'config': {'dtype': 'float32',
    'name': 'dropout_7',
    'noise_shape': None,
    'rate': 0.2,
    'seed': None,
    'trainable': True}},
  {'class_name': 'Dense',
   'config': {'activation': 'linear',
    'activity_regularizer': None,
    'bias_constraint'

In [37]:
model.to_json()

'{"class_name": "Sequential", "config": {"name": "sequential_7", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 784], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_14_input"}}, {"class_name": "Dense", "config": {"name": "dense_14", "trainable": true, "batch_input_shape": [null, 784], "dtype": "float32", "units": 512, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dropout", "config": {"name": "dropout_7", "trainable": true, "dtype": "float32", "rate": 0.2, "noise_shape": null, "seed": null}}, {"class_name": "Dense", "config": {"name": "dense_15", "trainable": true, "dtype": "float32", "units": 10, "activation": "linear", "use_bias": true, "kernel_init